In [44]:
import pandas as pd
import numpy as np

In [45]:
df = pd.read_csv('C:/Users/user/공공빅데이터_프로젝트/통합QGIS처리데이터.csv', encoding = "EUC-KR")
df

,id,lon,lat,주유소개수,주차장개수,도시공원개수
0,14939,129.025716,35.157394,3,0,0
1,11380,128.962523,35.123010,2,0,0
2,11391,128.961992,35.093263,2,0,0
3,11582,128.965137,35.085110,2,0,0
4,11770,128.968427,35.085071,2,0,0
...,...,...,...,...,...,...
1016,18131,129.081901,35.167493,0,0,0
1017,18133,129.081796,35.162084,0,0,0
1018,18134,129.081744,35.159380,0,0,0
1019,18135,129.081692,35.156676,0,0,0


In [46]:
# 칼럼확인
all_columns = df.columns.tolist()
all_columns

['id', 'lon', 'lat', '주유소개수', '주차장개수', '도시공원개수']

In [47]:
점수칼럼 =  ['주유소개수', '주차장개수','도시공원개수']

In [48]:
print("점수컬럼명\n", 점수칼럼)

점수컬럼명
 ['주유소개수', '주차장개수', '도시공원개수']


# 가중치선언

In [49]:
가중치 = [5,1,2]

if (len(가중치) != len(점수칼럼)):
    print("입력한 가중치 갯수가 점수칼럼 갯수와 다르므로 확인후 다시 입력 바랍니다")
else:
    print("다음 줄을 실행 하세요")

다음 줄을 실행 하세요


In [50]:
df.fillna(0, inplace=True)
df

,id,lon,lat,주유소개수,주차장개수,도시공원개수
0,14939,129.025716,35.157394,3,0,0
1,11380,128.962523,35.123010,2,0,0
2,11391,128.961992,35.093263,2,0,0
3,11582,128.965137,35.085110,2,0,0
4,11770,128.968427,35.085071,2,0,0
...,...,...,...,...,...,...
1016,18131,129.081901,35.167493,0,0,0
1017,18133,129.081796,35.162084,0,0,0
1018,18134,129.081744,35.159380,0,0,0
1019,18135,129.081692,35.156676,0,0,0


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021 entries, 0 to 1020
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      1021 non-null   int64  
 1   lon     1021 non-null   float64
 2   lat     1021 non-null   float64
 3   주유소개수   1021 non-null   int64  
 4   주차장개수   1021 non-null   int64  
 5   도시공원개수  1021 non-null   int64  
dtypes: float64(2), int64(4)
memory usage: 48.0 KB


In [52]:
# 총점계산
total_score = df.copy()

for i in range(0, len(점수칼럼)):
    name = 점수칼럼[i]
    df[name] = df[name] * 가중치[i] 

df.set_index(['id','lon','lat'], inplace=True)

df['sum'] = df.sum(1)
df.reset_index(inplace=True)

In [53]:
# 등수 계산 
df['순위'] = df['sum'].rank(method='dense', ascending=False).astype(int)

final = df.sort_values(by='순위')
final.reset_index(drop=True, inplace=True)

In [54]:
# 상위 20개만 학인
display("결과", final.head(20))

'결과'

,id,lon,lat,주유소개수,주차장개수,도시공원개수,sum,순위
0,14939,129.025716,35.157394,15,0,0,15,1
1,17940,129.078765,35.175648,10,0,2,12,2
2,12710,128.984876,35.084870,10,0,2,12,2
3,12894,128.988363,35.095647,0,1,10,11,3
4,11969,128.971183,35.055283,10,1,0,11,3
5,13274,128.994746,35.084749,10,0,0,10,4
6,14377,129.015737,35.152110,10,0,0,10,4
7,11770,128.968427,35.085071,10,0,0,10,4
8,11582,128.965137,35.085110,10,0,0,10,4
9,11391,128.961992,35.093263,10,0,0,10,4


In [55]:
지도보기갯수 = 5

In [56]:
# 지도로 보여주기
! pip install folium
import folium
print(f"보고자 하는 상위권 갯수는 {지도보기갯수}개 입니다.")
for i in range(지도보기갯수):
    지도보기 = folium.Map(location=[final.loc[i,'lat'],final.loc[i,'lon']], zoom_start = 15)
    folium.Marker([final.loc[i,'lat'],final.loc[i,'lon']]).add_to(지도보기)
    print (final.loc[i,'lat'],final.loc[i,'lon'])
    j = i+1
    display(f'상위 {j}번째 격자의 센터 위치 보기', 지도보기)

보고자 하는 상위권 갯수는 5개 입니다.
35.15739417 129.0257159


'상위 1번째 격자의 센터 위치 보기'

35.17564803 129.0787651


'상위 2번째 격자의 센터 위치 보기'

35.08487024 128.9848764


'상위 3번째 격자의 센터 위치 보기'

35.09564701 128.9883629


'상위 4번째 격자의 센터 위치 보기'

35.05528333 128.9711827


'상위 5번째 격자의 센터 위치 보기'